# ライブラリインポート

In [7]:
from PIL import Image
from io import BytesIO
import numpy as np
import glob
import matplotlib.pyplot as plt
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation,Dropout,Flatten,Dense,Input
from keras.utils  import np_utils

ImportError: cannot import name 'multi_worker_mirrored_2x1_gpu_noshare' from 'tensorflow.python.distribute.strategy_combinations' (C:\Users\toshi\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\distribute\strategy_combinations.py)

# データセット読み込み

In [ ]:
dataset_dir_name = "./input_images/img_align_celeba_png"
# dataset_dir_name = "./input_images/testimages"
dataset_fil_name = dataset_dir_name+"/*.png"
image_resize = (256, 256)
txtdata_fil_name = "./input_images/list_attr_celeba.txt"

X_train = []
dataset = glob.glob(dataset_fil_name)
for i,filename in enumerate(dataset):
    image_array = list(np.asarray(Image.open(filename).convert("RGB").resize(image_resize)))
    X_train.append(image_array)
    if i==3000:
        break
X_train = np.array(X_train)

Y_train = []
f = open(txtdata_fil_name)
f.readline()
f.readline()
for i in range(len(X_train)):
    y = int(f.readline().split()[32])
    if y==1:
        Y_train.append([1,0])
    else:
        Y_train.append([0,1])
f.close()
Y_train = np.array(Y_train)
print(np.shape(X_train))


# モデル作成

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation="relu",padding="same",input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),padding="same"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(2, activation="softmax"))
model.compile(loss="categorical_crossentropy")
model.summary()

# モデル実行

In [ ]:
history = model.fit(X_train, Y_train,validation_split=0.2, epochs=30)
plt.plot(history.history['loss']) # 損失関数
plt.plot(history.history['val_loss']) # 損失関数
plt.title('Model loss') 
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
prediction = model.predict(X_train[:30])  

for p in prediction:
    if p[0]>p[1]:
        print("smile")
    else:
        print("not smile")

In [ ]:
model.save("./model.h5")

In [ ]:
!C:\Users\toshi\AppData\Roaming\Python\Python310\Scripts\tensorflowjs_converter.exe --input_format keras ./model.h5 ./